In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
ls data/sports_scores/

astros_1A10.csv*   Rice_10H80.csv*    Tex_15E40.csv*
dynamo_10H30.csv*  rockets_1A10.csv*  UH_10H50.csv*


In [3]:
ls data/crime_beat/

crime_10H30_10_17.csv*  crime_10H80_10_17.csv*  crime_1A10_10_17.csv*
crime_10H50_10_17.csv*  crime_15E40_10_17.csv*


In [4]:
games = 'data/sports_scores/Tex_15E40.csv'
crime = 'data/crime_beat/crime_15E40_10_17.csv'

# scores df

In [5]:
df = pd.read_csv(games)


## split date and time from column scheduled
s_date = pd.DatetimeIndex(df['scheduled'])
df['Date'] = s_date.date
df['time'] = s_date.time
del df['scheduled']

# set date as datetime and index
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date').sort_index(ascending=True)






## win column
df['win'] =  np.where(df['scoring.home_points'] > df['scoring.away_points'],1,0)

# only oposing name and win boolean
df = df[['away.alias','win']]

# show

df.head()

,away.alias,win
Date,,
2010-01-03,NE,1
2010-09-12,IND,1
2010-09-26,DAL,0
2010-10-10,NYG,0
2010-10-17,KC,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 69 entries, 2010-01-03 to 2017-12-25
Data columns (total 2 columns):
away.alias    69 non-null object
win           69 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


# Crime df

In [7]:
crime_df = pd.read_csv(crime).sort_index(ascending=True)

crime_df['Date_Hour'] = pd.to_datetime(crime_df['Date_Hour'])
crime_df['Date_Hour'] = crime_df['Date_Hour'].dt.date

crime_df = crime_df.set_index('Date_Hour')


# selectd columns
crime_df = crime_df[['OffenseType']]

# count Offense type given date
date_crimes = crime_df.groupby(crime_df.index)['OffenseType',].count()

## show
date_crimes.head()

,OffenseType
Date_Hour,
2010-01-01,2
2010-01-02,2
2010-01-03,5
2010-01-04,2
2010-01-05,4


In [8]:
#crime_df = pd.get_dummies(crime_df,drop_first=True)

In [9]:
#crime_df.info()

# Days Df
- create a df with everyday timestamp from Jan 2010 to Dec 2017

In [10]:
days = pd.date_range(start='01/01/2010', end='12/31/2017')
days = pd.DataFrame(days)

days.columns = ['days']
days = days.set_index('days').sort_index(ascending=True)
days.head()

""
days
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


In [11]:
365 * 8

2920

In [12]:
len(days)/365

8.005479452054795

In [13]:
days.tail()

""
days
2017-12-27
2017-12-28
2017-12-29
2017-12-30
2017-12-31


# Merge days df with crimes_df

In [14]:
calendar_crimes = pd.merge(days,date_crimes,  left_index=True, right_index=True, how='left')
calendar_crimes.head()

,OffenseType
days,
2010-01-01,2.0
2010-01-02,2.0
2010-01-03,5.0
2010-01-04,2.0
2010-01-05,4.0


In [15]:
calendar_crimes.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 1 columns):
OffenseType    2886 non-null float64
dtypes: float64(1)
memory usage: 125.7 KB


In [16]:
calendar_crimes.OffenseType.value_counts(dropna=False)

 5.0     420
 4.0     403
 6.0     393
 3.0     366
 7.0     322
 2.0     229
 8.0     220
 9.0     161
 1.0     105
 10.0     84
 11.0     72
 12.0     42
NaN       36
 13.0     28
 14.0     12
 16.0      7
 15.0      6
 19.0      5
 17.0      4
 20.0      2
 21.0      1
 26.0      1
 32.0      1
 18.0      1
 22.0      1
Name: OffenseType, dtype: int64

## Change nans with 0

In [17]:
calendar_crimes.OffenseType.replace( np.nan,0, inplace=True)

In [18]:
calendar_crimes.OffenseType.value_counts(dropna=False)

5.0     420
4.0     403
6.0     393
3.0     366
7.0     322
2.0     229
8.0     220
9.0     161
1.0     105
10.0     84
11.0     72
12.0     42
0.0      36
13.0     28
14.0     12
16.0      7
15.0      6
19.0      5
17.0      4
20.0      2
21.0      1
26.0      1
32.0      1
18.0      1
22.0      1
Name: OffenseType, dtype: int64

# merge calendar_crimes with score

In [19]:
merge_data = pd.merge(calendar_crimes,df,  left_index=True, right_index=True, how='left')

# change column names
#merge_data.columns = ['offenses','away_team','win']

merge_data.head()

,OffenseType,away.alias,win
days,,,
2010-01-01,2.0,NaN,NaN
2010-01-02,2.0,NaN,NaN
2010-01-03,5.0,NE,1.0
2010-01-04,2.0,NaN,NaN
2010-01-05,4.0,NaN,NaN


In [20]:
merge_data['away.alias'].value_counts(dropna=False)

NaN    2853
TEN       8
IND       8
JAC       8
KC        5
CIN       4
BAL       3
NE        3
OAK       3
SD        2
PIT       2
BUF       2
CLE       2
ARI       1
TB        1
CHI       1
MIA       1
NO        1
CAR       1
ATL       1
SF        1
MIN       1
GB        1
NYJ       1
STL       1
PHI       1
NYG       1
DET       1
DAL       1
SEA       1
WAS       1
DEN       1
Name: away.alias, dtype: int64

## Cleanup colums

In [21]:
merge_data.columns  = ['crime', 'away_team', 'won']

In [22]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2922 non-null float64
away_team    69 non-null object
won          69 non-null float64
dtypes: float64(2), object(1)
memory usage: 171.3+ KB


In [23]:
merge_data.crime.value_counts(dropna=False)

5.0     420
4.0     403
6.0     393
3.0     366
7.0     322
2.0     229
8.0     220
9.0     161
1.0     105
10.0     84
11.0     72
12.0     42
0.0      36
13.0     28
14.0     12
16.0      7
15.0      6
19.0      5
17.0      4
20.0      2
21.0      1
26.0      1
32.0      1
18.0      1
22.0      1
Name: crime, dtype: int64

In [24]:
merge_data.crime = merge_data.crime.astype(int)

In [25]:
merge_data.crime.value_counts(dropna=False)

5     420
4     403
6     393
3     366
7     322
2     229
8     220
9     161
1     105
10     84
11     72
12     42
0      36
13     28
14     12
16      7
15      6
19      5
17      4
20      2
21      1
18      1
22      1
32      1
26      1
Name: crime, dtype: int64

In [26]:
merge_data

,crime,away_team,won
days,,,
2010-01-01,2,NaN,NaN
2010-01-02,2,NaN,NaN
2010-01-03,5,NE,1.0
2010-01-04,2,NaN,NaN
2010-01-05,4,NaN,NaN
2010-01-06,6,NaN,NaN
2010-01-07,3,NaN,NaN
2010-01-08,6,NaN,NaN
2010-01-09,5,NaN,NaN


# Display non-nans

In [27]:
merge_data.dropna(subset=['away_team', 'won'])

,crime,away_team,won
days,,,
2010-01-03,5,NE,1.0
2010-09-12,7,IND,1.0
2010-09-26,13,DAL,0.0
2010-10-10,5,NYG,0.0
2010-10-17,6,KC,1.0
2010-11-07,4,SD,0.0
2010-11-28,9,TEN,1.0
2010-12-14,5,BAL,0.0
2011-01-02,6,JAC,1.0


In [28]:
merge_data.to_csv('data/final/Texans1017_final.csv')

In [29]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2922 non-null int64
away_team    69 non-null object
won          69 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 171.3+ KB
